In [2]:
import torch

In [ ]:
GPT_CONFIG_124M={
    'vocab_size':50257,
    'context_length':1024,
    'emb_dim':768,
    'n_head':12,
    'n_layers':12,
    'drop_rate':0.1,
    'qkv_bias':False
}

In [5]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [27]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [32]:
class CausalSelfAttention(torch.nn.Module):
    
    
    

    def __init__(self,in_dim,out_dim,context_length,drop_rate):
        super().__init__()
        self.kw=torch.nn.Linear(in_dim,out_dim)
        print(self.kw)
        self.qw=torch.nn.Linear(in_dim,out_dim)
        self.vw=torch.nn.Linear(in_dim,out_dim)
        self.dropout=torch.nn.Dropout(drop_rate)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    
    def forward(self,x):
        b,t,in_dim=x.shape

        key=self.kw(x)
        print(key)
        query=self.qw(x)
        value=self.vw(x)

        attn_score=query@key.transpose(1,2)

        attn_score.masked_fill_(self.mask.bool()[:t,:t],-torch.inf)
        print(attn_score)
        attn_weight=torch.softmax(attn_score/key.shape[-1]**0.5,dim=-1)
        print(attn_weight)
        
        attn_weight=self.dropout(attn_weight)
        
        context_vectore=attn_weight@value


        print(context_vectore)

        return context_vectore



x=CausalSelfAttention(batch.shape[-1],6,batch.shape[1],0.3)
x(batch)





Linear(in_features=3, out_features=6, bias=True)
tensor([[[-0.9083,  0.6658, -0.4917, -0.2189, -0.3603, -0.5090],
         [-0.4897,  0.3671, -0.4970, -0.1402, -0.1464, -0.8430],
         [-0.4839,  0.3517, -0.4892, -0.1401, -0.1715, -0.8444],
         [-0.4565,  0.4113, -0.3878,  0.0363, -0.1990, -0.5902],
         [-0.3892,  0.1035, -0.2814, -0.0513, -0.7003, -0.7090],
         [-0.5150,  0.5579, -0.4721,  0.0220,  0.0509, -0.5969]],

        [[-0.9083,  0.6658, -0.4917, -0.2189, -0.3603, -0.5090],
         [-0.4897,  0.3671, -0.4970, -0.1402, -0.1464, -0.8430],
         [-0.4839,  0.3517, -0.4892, -0.1401, -0.1715, -0.8444],
         [-0.4565,  0.4113, -0.3878,  0.0363, -0.1990, -0.5902],
         [-0.3892,  0.1035, -0.2814, -0.0513, -0.7003, -0.7090],
         [-0.5150,  0.5579, -0.4721,  0.0220,  0.0509, -0.5969]]],
       grad_fn=<ViewBackward0>)
tensor([[[ 0.1007,    -inf,    -inf,    -inf,    -inf,    -inf],
         [ 0.4958,  0.0582,    -inf,    -inf,    -inf,    -inf],
     

tensor([[[ 0.0650,  0.4974, -1.0731,  0.5541, -0.1693,  0.5412],
         [-0.1948, -0.0841, -0.5629,  0.0543, -0.2391,  0.4749],
         [-0.1097,  0.1279, -0.7878,  0.2442, -0.2276,  0.5287],
         [-0.3105,  0.0014, -1.0834,  0.2069, -0.3874,  0.8901],
         [-0.3780, -0.0176, -1.0250,  0.2195, -0.3868,  0.8606],
         [-0.3840, -0.1352, -0.8097,  0.0670, -0.3519,  0.7973]],

        [[ 0.0650,  0.4974, -1.0731,  0.5541, -0.1693,  0.5412],
         [-0.1594,  0.1868, -1.1472,  0.3560, -0.3313,  0.7696],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [-0.3281, -0.1330, -0.7934,  0.0572, -0.3417,  0.7438],
         [-0.2204, -0.0504, -0.3107,  0.0531, -0.1435,  0.3357],
         [-0.2804, -0.1221, -0.6816,  0.0222, -0.2906,  0.6775]]],
       grad_fn=<UnsafeViewBackward0>)

In [39]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self,in_dim,out_dim,context_length,drop_rate,num_head):
        self.head=torch.nn.ModuleList([CausalSelfAttention(in_dim,out_dim,context_length,drop_rate) for _ in range(num_head)])
        print(self.head)
    def forward(self,x):
        print(self.head)
        return torch.cat([head(x) for head in self.head],dim=-1)
    
x=MultiHeadAttention(batch.shape[-1],6,batch.shape[1],0.3,3)
x(batch)

Linear(in_features=3, out_features=6, bias=True)
Linear(in_features=3, out_features=6, bias=True)
Linear(in_features=3, out_features=6, bias=True)


AttributeError: cannot assign module before Module.__init__() call

In [ ]:
class GPTModel(torch.nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb=torch.nn.Embedding(cfg['vocab_size'],cfg['emb_dim'])
        self.pos_emb=torch.nn.Embedding(cfg['context_length'],cfg['emb_dim'])
        self.drop_emb=torch.nn.Dropout(cfg['drop_rate'])

        self.trf_blocks=torch.nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])

        self.final_norm=LayerNorm(cfg['emb_dim'])#implemented
        self.output_head=torch.nn.Linear(cfg['emb_dim'], cfg['vocab_size'])

    
    def forward(self,in_idx):#input ID
        batch_size,seq_len=in_idx.shape()
        tok_emb=self.tok_emb(in_idx)
        pos_emb=self.pos_emb(torch.arange(seq_len,device=in_idx.device))
        x=tok_emb+pos_emb
        x=self.drop_emb(x)
        x=self.trf_blocks(x)#transformerblock implemented
        logits=self.output_head(x)

        return logits